In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Sep  2 01:31:23 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:0A:00.0  On |                  N/A |
|  0%   35C    P5    52W / 390W |   1231MiB / 24576MiB |     34%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
#%%capture
#!pip install datasets==1.13.3
#!pip install transformers==4.11.3
#!pip install huggingface_hub==0.1
#!pip install torchaudio==0.10.0+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
#!pip install jiwer

In [3]:
from datasets import load_dataset, load_metric, Audio

common_voice_train = load_dataset("mozilla-foundation/common_voice_10_0", "uk", split="train", use_auth_token=True)
common_voice_test = load_dataset("mozilla-foundation/common_voice_10_0", "uk", split="test", use_auth_token=True)

Reusing dataset common_voice_10_0 (/home/robinhad/.cache/huggingface/datasets/mozilla-foundation___common_voice_10_0/uk/10.0.0/27df768ab1b5cac48a7616f145b79b62599167b0ffa2e054bf4c3e74e9619e5e)
Reusing dataset common_voice_10_0 (/home/robinhad/.cache/huggingface/datasets/mozilla-foundation___common_voice_10_0/uk/10.0.0/27df768ab1b5cac48a7616f145b79b62599167b0ffa2e054bf4c3e74e9619e5e)


In [4]:
common_voice_train

Dataset({
    features: ['client_id', 'path', 'audio', 'sentence', 'up_votes', 'down_votes', 'age', 'gender', 'accent', 'locale', 'segment'],
    num_rows: 11463
})

In [5]:
common_voice_train.cleanup_cache_files()
common_voice_test.cleanup_cache_files()

0

In [6]:
common_voice_train = common_voice_train.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])
common_voice_test = common_voice_test.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "segment", "up_votes"])

In [7]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [8]:
show_random_elements(common_voice_train.remove_columns(["path", "audio"]), num_examples=10)

,sentence
0,Вона нас не лякає.
1,"Бейнбрідж затримався, готуючи екіпажі, й фактично не встиг узяти участі у війні."
2,А тепер.
3,"Наші ""будьонівці"", ніби з цікавості, зібралися й оточили червоні шеренги."
4,"Серед квітів я вмирав, Серед хмар я воскресав."
5,Сьогодні виробництво полімерів найбільша галузь хімічної промисловості.
6,Хмельницький заплатив за все на цілий рік наперед.
7,Соловій же залишився підпалити бікфордів шнур.
8,"Тоді його слово буде хвилювати, захоплювати, піднімати людську душу."
9,"Тут були яблуні, сливи, вишні, — вишень найбільше."


In [9]:
filter_func = lambda x: not ("joki" in x or "ы" in x)
common_voice_train = common_voice_train.filter(filter_func, input_columns=["sentence"])
common_voice_test = common_voice_test.filter(filter_func, input_columns=["sentence"])

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/7 [00:00<?, ?ba/s]

In [10]:
def cleaner(batch):
    replace_as_space = "!:;,—…–“”?\"«»"
    special_words = {
        "ХIХ": "дев'ятнадцятого",
        "Linux": "Лінукс",
        "Maace": "Маасе",
        "м 'ясо": "м'ясо",
        "'іде": "іде",
        "Д'Аламбер": "даламбер",
        " - ": " ",
        "--": " ",
        "....": " ",
        "...": " ",
        "..": " ",
        " '": " ",
        "О'": "о",
        "-": "" #further check needed
    }
    # check abbreviations later
    abbreviations = {
        'ЧК': "чека",
        'ҐПУ': "ґепеу",
        'ЄС.': "єес",
        'УНР': "уенер",
        'ДТП.': "детепе",
        'РНБО': "еренбео",
        'СРСР': "есересер",
        'ДБР': "дебеер",
        'КП': "капе",
        'ОС': "оес",
    } 
    chars_dict = {
        "C": "С",
        "I": "І",
        "P": "Р",
        "a": "а",
        "e": "е",
        "x": "х",
        "y": "у",
        "p": "р",
        "o": "о",
        "i": "і",
        "\u0301": "",
        "`": "'",
        "՚": "'",
        ".": " ",
        "’": "'"
        
    }
    for word in special_words.keys():
        batch["sentence"] = batch["sentence"].replace(word, special_words[word])
    for word in abbreviations.keys():
        batch["sentence"] = batch["sentence"].replace(word, abbreviations[word])
    for char in chars_dict.keys():
        batch["sentence"] = batch["sentence"].replace(char, chars_dict[char])
    for char in replace_as_space:
        batch["sentence"] = batch["sentence"].replace(char, " ")
    batch["sentence"] = " ".join(filter(lambda x: x != "", batch["sentence"].strip().lower().split(" ")))
    return batch

sentence = {"sentence": "Привіт, - як у тебе справи загалом, м'якотілий друже?"}
cleaner(sentence)

{'sentence': "привіт як у тебе справи загалом м'якотілий друже"}

In [11]:
show_random_elements(common_voice_train.map(cleaner).remove_columns(["path", "audio"]), num_examples=10)

  0%|          | 0/11463 [00:00<?, ?ex/s]

,sentence
0,так так усе на світі кінчається
1,комуністів тільки й є що воєнком та два ротні політруки
2,уже й убитих чимало
3,трупів не закопували
4,до фастова дісталася з якимось польським обозом
5,невже то ви були
6,при отій купці отої нещасної духовної братії
7,вирішуємо напасти на бригаду по дорозі не допустивши до села
8,де то хто таке видав аби хлопи купували панські маєтки
9,коні пішли з коноводами в балку


In [12]:
common_voice_train = common_voice_train.map(cleaner)
common_voice_test = common_voice_test.map(cleaner)

Loading cached processed dataset at /home/robinhad/.cache/huggingface/datasets/mozilla-foundation___common_voice_10_0/uk/10.0.0/27df768ab1b5cac48a7616f145b79b62599167b0ffa2e054bf4c3e74e9619e5e/cache-890587fbc5f83609.arrow


  0%|          | 0/6783 [00:00<?, ?ex/s]

In [13]:
show_random_elements(common_voice_train.remove_columns(["path","audio"]))

,sentence
0,я замилувався маневруванням тачанок на полях
1,андрій із бугаєм вилізли на близький горб роздивилися
2,вона нам потрібна як щоденний хліб
3,олесеві ще більше захотілось чаю
4,вирішуємо тут поснідати і з годину відпочити бо люди й коні потомлені
5,воротилову сотню найдужче боявся він дрібного дощу який почав сіятись удосвіта й міг зашкодити
6,люта злість піднялася в душі хлопця при вигляді оцього свого тирана
7,також цього року в столиці виникла низка профспілкових організацій і був створений робочий клуб
8,накидав того літа а вони в наших плавнях затрималися всю січ мені засмерділи
9,їздять коло нас


In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["sentence"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocab_train = common_voice_train.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_train.column_names)
vocab_test = common_voice_test.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=common_voice_test.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [17]:
vocab_dict = {v: k for k, v in enumerate(sorted(vocab_list))}
vocab_dict

{' ': 0,
 "'": 1,
 'а': 2,
 'б': 3,
 'в': 4,
 'г': 5,
 'д': 6,
 'е': 7,
 'ж': 8,
 'з': 9,
 'и': 10,
 'й': 11,
 'к': 12,
 'л': 13,
 'м': 14,
 'н': 15,
 'о': 16,
 'п': 17,
 'р': 18,
 'с': 19,
 'т': 20,
 'у': 21,
 'ф': 22,
 'х': 23,
 'ц': 24,
 'ч': 25,
 'ш': 26,
 'щ': 27,
 'ь': 28,
 'ю': 29,
 'я': 30,
 'є': 31,
 'і': 32,
 'ї': 33,
 'ґ': 34}

In [18]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [19]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

37

In [20]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [21]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer.from_pretrained("./", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [22]:
repo_name = "wav2vec2-xls-r-300m-uk"

In [23]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [24]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [25]:
# save tokenizer to folder
processor.save_pretrained(repo_name)

In [26]:
common_voice_train[0]["path"]

'/home/robinhad/.cache/huggingface/datasets/downloads/extracted/ee7155196e5d51620d53e48cf58eb693b7839b8ff183604c8bb948d3e0aad92d/cv-corpus-10.0-2022-07-04/uk/clips/common_voice_uk_20907128.mp3'

In [27]:
common_voice_train[0]["audio"]

{'path': '/home/robinhad/.cache/huggingface/datasets/downloads/extracted/ee7155196e5d51620d53e48cf58eb693b7839b8ff183604c8bb948d3e0aad92d/cv-corpus-10.0-2022-07-04/uk/clips/common_voice_uk_20907128.mp3',
 'array': array([ 0.0000000e+00, -3.5002383e-14,  9.4785833e-15, ...,
        -5.0386465e-08, -4.4114326e-08, -1.9402206e-08], dtype=float32),
 'sampling_rate': 48000}

In [28]:
common_voice_train = common_voice_train.cast_column("audio", Audio(sampling_rate=16_000))
common_voice_test = common_voice_test.cast_column("audio", Audio(sampling_rate=16_000))

In [29]:
common_voice_train[0]["audio"]

{'path': '/home/robinhad/.cache/huggingface/datasets/downloads/extracted/ee7155196e5d51620d53e48cf58eb693b7839b8ff183604c8bb948d3e0aad92d/cv-corpus-10.0-2022-07-04/uk/clips/common_voice_uk_20907128.mp3',
 'array': array([ 1.00456624e-13, -1.54340042e-13,  7.00158518e-13, ...,
        -1.50335762e-08, -1.92623926e-08, -2.21930367e-08], dtype=float32),
 'sampling_rate': 16000}

In [30]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(common_voice_train)-1)

print(common_voice_train[rand_int]["sentence"])
ipd.Audio(data=common_voice_train[rand_int]["audio"]["array"], autoplay=True, rate=16000)

от би була рада


In [31]:
rand_int = random.randint(0, len(common_voice_train)-1)

print("Target text:", common_voice_train[rand_int]["sentence"])
print("Input array shape:", common_voice_train[rand_int]["audio"]["array"].shape)
print("Sampling rate:", common_voice_train[rand_int]["audio"]["sampling_rate"])

Target text: от їхало якихось із десять на конях назустріч
Input array shape: (73152,)
Sampling rate: 16000


In [32]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched"
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["sentence"]).input_ids
    return batch

In [33]:
common_voice_train = common_voice_train.map(prepare_dataset, remove_columns=common_voice_train.column_names)
common_voice_test = common_voice_test.map(prepare_dataset, remove_columns=common_voice_test.column_names)

  0%|          | 0/11463 [00:00<?, ?ex/s]

  0%|          | 0/6783 [00:00<?, ?ex/s]

In [34]:
#max_input_length_in_sec = 5.0
#common_voice_train = common_voice_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

In [35]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [36]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [37]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")
metrics = [wer_metric, cer_metric]

In [38]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

In [39]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-xls-r-300m", 
    attention_dropout=0.3,
    hidden_dropout=0.3,
    feat_proj_dropout=0.3,
    mask_time_prob=0.05,
    layerdrop=0.3,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer),
)

Some weights of the model checkpoint at facebook/wav2vec2-xls-r-300m were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'project_hid.bias', 'project_q.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_hid.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it 

In [40]:
model.freeze_feature_extractor()

/home/robinhad/Projects/Speech/wav2vec2-xls-r-ukrainian/.venv/lib/python3.9/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1618: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [41]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=16,
  gradient_accumulation_steps=6,
  eval_accumulation_steps=4,
  evaluation_strategy="steps",
  num_train_epochs=100,
  gradient_checkpointing=True,
  fp16=True,
  save_steps=400,
  eval_steps=400,
  logging_steps=400,
  learning_rate=3e-4,
  warmup_steps=500,
  save_total_limit=2,
  report_to="tensorboard"
)

In [43]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=common_voice_train,
    eval_dataset=common_voice_test,
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [44]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/home/robinhad/Projects/Speech/wav2vec2-xls-r-ukrainian/.venv/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11463
  Num Epochs = 100
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 96
  Gradient Accumulation steps = 6
  Total optimization steps = 11900


  0%|          | 0/11900 [00:00<?, ?it/s]

The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 6783
  Batch size = 8


{'loss': 5.4923, 'learning_rate': 0.0002388, 'epoch': 3.36}


  0%|          | 0/848 [00:00<?, ?it/s]

...

Saving model checkpoint to wav2vec2-xls-r-300m-uk/checkpoint-11600
Configuration saved in wav2vec2-xls-r-300m-uk/checkpoint-11600/config.json


{'eval_loss': 0.9188246130943298, 'eval_wer': 0.660722256035707, 'eval_cer': 0.17280426731416873, 'eval_runtime': 211.3614, 'eval_samples_per_second': 32.092, 'eval_steps_per_second': 4.012, 'epoch': 97.48}


Model weights saved in wav2vec2-xls-r-300m-uk/checkpoint-11600/pytorch_model.bin
Feature extractor saved in wav2vec2-xls-r-300m-uk/checkpoint-11600/preprocessor_config.json
Deleting older checkpoint [wav2vec2-xls-r-300m-uk/checkpoint-10800] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 40015.4, 'train_samples_per_second': 28.646, 'train_steps_per_second': 0.297, 'train_loss': 0.9631941716811236, 'epoch': 100.0}


TrainOutput(global_step=11900, training_loss=0.9631941716811236, metrics={'train_runtime': 40015.4, 'train_samples_per_second': 28.646, 'train_steps_per_second': 0.297, 'train_loss': 0.9631941716811236, 'epoch': 100.0})

In [ ]:
trainer.create_model_card()

Dropping the following result as it does not have all the necessary fields:
{'dataset': {'name': 'common_voice', 'type': 'common_voice', 'args': 'uk'}}


In [ ]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
model = Wav2Vec2ForCTC.from_pretrained(repo_name + "/checkpoint-11200").to("cuda")
processor = Wav2Vec2Processor.from_pretrained(repo_name)

loading configuration file wav2vec2-xls-r-300m-uk/checkpoint-5600/config.json
Model config Wav2Vec2Config {
  "_name_or_path": "facebook/wav2vec2-xls-r-300m",
  "activation_dropout": 0.0,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 768,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropo


Now, we will just take the first example of the test set, run it through the model and take the `argmax(...)` of the logits to retrieve the predicted token ids.

In [ ]:
audio_id = 10

input_dict = processor(common_voice_test["input_values"], return_tensors="pt", padding=True)

logits = model(input_dict.input_values.to("cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)[audio_id]

common_voice_test_transcription = load_dataset("common_voice", "uk", split="test")

print("Prediction:")
print(processor.decode(pred_ids))

print("\nReference:")
print(common_voice_test_transcription[audio_id]["sentence"].lower())

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
